# Single Slew M1M3 Inertia Compensation System Analysis

This notebook is part of [SITCOMTN-092].  
Use it to analyze a single slew.  

The bulk analysis has been moved to [lsst-sitcom/summit_utils]  
and it is already available in the `main` branch.    
Refer to the [README.md] file for details on how to set up this repository in your environment.

  
[lsst-sitcom/summit_utils]: https://github.com/lsst-sitcom/summit_utils
[README.md]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[SITCOM-989]: https://jira.lsstcorp.org/browse/SITCOM-989
[SITCOMTN-092]: https://sitcomtn-092.lsst.io/

## Notebook Preparation

In [ ]:
day_obs = 20250527
slew_id = 37
tma_event_version = 0

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time
from pathlib import Path

from lsst.summit.utils.m1m3 import inertia_compensation_system as ics
from lsst.summit.utils.m1m3.plots import plot_ics
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)
from lsst.summit.utils.utils import setupLogging

setupLogging()

## Create Event Maker

We want to create a single instance of the `TMAEventMaker` object.  
Each instance might be quite heavy. 

In [ ]:
plot_path = Path("./plots")
plot_path.mkdir(exist_ok=True, parents=True)

tma_event_maker = TMAEventMaker()
efd_client = makeEfdClient()

## Helper functions

In [ ]:
def analyze_m1m3_ics_slew_event(begin, end, event_maker, log, path=None):
    """
    Plot the ICS performance analysis in a single slew.
    Three axes are created. The top representes the hard point forces.
    The second shows the velocity in azimuth and elevation.
    The thierd shows the torques in azimuth and elevation.

    Parameters
    ----------
    begin : str
        Approximate time of when the slew began in UTC using iso format.
    end : str
        Approximate time of when the slew ended in UTC using iso format.
    event_maker :
        TMA event maker
    log :
        Logger
    path : Path, optional
        Path to store plots
    """
    time_begin = Time(begin, format="isot", scale="utc")
    time_end = Time(end, format="isot", scale="utc")
    time_half = time_begin + (time_end - time_begin) * 0.5

    event = event_maker.findEvent(time_half)
    print(
        f"Slew happened from {begin=} to {end=} "
        f"and has sequence number {event.seqNum} "
        f"and observation day {event.dayObs}"
    )

    data = ics.M1M3ICSAnalysis(event, event_maker.client)
    name = f"ics_performance_ics_hp{data.stats.ics_enabled}_{data.stats.day_obs}_sn{data.stats.seq_num}_v{data.stats.version}"

    commands = getCommandsDuringEvent(
        event_maker.client, event, hardpoint_commands_to_plot
    )

    print(commands)
    
    fig = plt.figure(num=name, figsize=(10, 5), dpi=120)
    fig = plot_ics.plot_hp_measured_data(
        data, log=data.log, fig=fig, commands=commands
    )

    if path:
        fig.savefig(str(path / f"{name}"))

    # plt.show()

    return data

## Analysis

### Get all slews in a day

This analysis assume that you already have done the following steps and that you already know what is the slew ID you want to plot. In RubinTV, this slew ID is called as Sequence Number. It is the first column on the left in the [TMA RubinTV Page].

[TMA RubinTV Page]: https://summit-lsp.lsst.codes/rubintv/summit/tma

In [ ]:
tma_events = tma_event_maker.getEvents(day_obs)
track_events = [e for e in tma_events if e.type == TMAState.TRACKING]
slew_events = [e for e in tma_events if e.type == TMAState.SLEWING]

print(
    f"Found {len(tma_events)} TMA Events on day_obs={day_obs}\n"
    f"  Number of tracking events: {len(track_events)}\n"
    f"  Number of slewing events: {len(slew_events)}\n"
)

We know from the plots in RubinTV for `20250527` that the plots associates with Slew 37 show that the forces applied on the hard points are higher than the operational limit, but are still lower than the fatigue or safety limimt. So let's use it for further analysis.

## Analyze M1M3 ICS per Slew Event



In [ ]:
event = tma_events[slew_id]

In [ ]:
print(
    f"Slew happened from {event.begin} to {event.end}\n"
    f"  and has sequence number {event.seqNum}\n"
    f"  and observation day {event.dayObs}\n"
)

data = ics.M1M3ICSAnalysis(event, tma_event_maker.client)
name = f"ics_performance_ics_hp{data.stats.ics_enabled}_{data.stats.day_obs}_sn{data.stats.seq_num}_v{data.stats.version}"

fig = plt.figure(num=name, figsize=(10, 5), dpi=120)
fig = plot_ics.plot_hp_measured_data(data, log=data.log, fig=fig)

if plot_path:
    fig.savefig(str(plot_path / f"{name}"))

plt.show()

## Backward compatible slew analysis

The following data is quite old and we collected them during the M1M3+M2 Surrogates test campaings.  
We are keeping them here to ensure that the analysis code is backward compatible.

In [ ]:
slew_data = {
    # First data obtained at 30% motion settings and ICS disabled
    "20230727_ics_False_30": dict(
        begin="2023-07-28T02:17:15", end="2023-07-28T02:17:55"
    ),
    # Second data obtained at 30% motion settings and ICS enabled
    "20230802_ics_True_30": dict(
        begin="2023-08-02T22:02:30", end="2023-08-02T22:04:00"
    ),
    # Third data obtained at 50% motion settings and ICS enabled
    "20230802_ics_True_50": dict(
        begin="2023-08-03T03:20:30", end="2023-08-03T03:21:20"
    ),
    # More recent data obtained at Full Performance and ICS enabled
    "20231129_ics_True_100": dict(
        begin="2023-11-30T08:46:44", end="2023-11-30T08:47:45"
    ),
}

hardpoint_commands_to_plot = [
    "lsst.sal.MTM1M3.command_setSlewFlag",
    "lsst.sal.MTM1M3.command_enableHardpointCorrections",
    "lsst.sal.MTM1M3.command_clearSlewFlag",
]

### Case 1 - ICS Disabled and 30% TMA Performance

In [ ]:
%matplotlib inline
data = analyze_m1m3_ics_slew_event(
    slew_data["20230727_ics_False_30"]["begin"],
    slew_data["20230727_ics_False_30"]["end"],
    tma_event_maker,
    plot_path,
)

data.stats

### Case 2 - ICS Enabled and 30% TMA Performance

In [ ]:
%matplotlib inline
data = analyze_m1m3_ics_slew_event(
    slew_data["20230802_ics_True_30"]["begin"],
    slew_data["20230802_ics_True_30"]["end"],
    tma_event_maker,
    plot_path,
)

print(data.stats)

### Case 3 - ICS Enabled and 50% TMA Performance

In [ ]:
%matplotlib inline
data = analyze_m1m3_ics_slew_event(
    slew_data["20230802_ics_True_50"]["begin"],
    slew_data["20230802_ics_True_50"]["end"],
    tma_event_maker,
    plot_path,
)

print(data.stats)

### Case 4 - ICS Enabled and 100% TMA Performance

In [ ]:
%matplotlib inline
data = analyze_m1m3_ics_slew_event(
    slew_data["20231129_ics_True_100"]["begin"],
    slew_data["20231129_ics_True_100"]["end"],
    tma_event_maker,
    plot_path,
)

print(data.stats)